In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv(r'E:\data set\test.csv')

df.head()

,Date,store,product,number_sold
0,2019-01-01,0,0,845
1,2019-01-02,0,0,851
2,2019-01-03,0,0,840
3,2019-01-04,0,0,842
4,2019-01-05,0,0,845


In [7]:
df.columns

Index(['Date', 'store', 'product', 'number_sold'], dtype='object')

In [8]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Extract features like year, month, and day from the 'Date' column
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

# Drop the original 'Date' column
df = df.drop('Date', axis=1)


Encode Categorical Variables
Use one-hot encoding for categorical columns like store and product

In [9]:
# One-hot encode categorical columns
df = pd.get_dummies(df, columns=['store', 'product'], drop_first=True)


Prepare Data for the Model

In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Separate features and target variable
features = df.drop('number_sold', axis=1).values
target = df['number_sold'].values

# Normalize features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Reshape for CNN input (assuming sequence length and features are known)
num_timesteps = 10  # Adjust according to your data
num_features = X_train.shape[1]

# Adjust the shapes accordingly
X_train = X_train[:(X_train.shape[0] // num_timesteps) * num_timesteps].reshape(-1, num_timesteps, num_features)
X_test = X_test[:(X_test.shape[0] // num_timesteps) * num_timesteps].reshape(-1, num_timesteps, num_features)


In [17]:
# Add a test layer to check the shape
model.add(tf.keras.layers.Lambda(lambda x: tf.print(tf.shape(x))))


 Build and Train the Hybrid Model

In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Reshape, LSTM, Dense
from tensorflow.keras.models import Model

# Parameters (update these as needed)
num_timesteps = 10
num_features = 5

# Define the model using the Functional API
inputs = Input(shape=(num_timesteps, num_features))

# Example Conv1D layer - adjust the kernel size and filters as needed
x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)

# Calculate the flattened shape after MaxPooling1D
flattened_shape = int(x.shape[1])  # Assuming batch size is None

# Determine new timesteps and features based on flattened shape
new_timesteps = 5
new_features = flattened_shape // new_timesteps

# Ensure divisibility for reshaping
if flattened_shape % new_timesteps != 0:
  raise ValueError("Input shape after MaxPooling must be divisible by new_timesteps.")

# Reshape with correct calculation
x = Reshape((new_timesteps, new_features))(x)
x = LSTM(50, return_sequences=False)(x)
outputs = Dense(1, activation='linear')(x)

# Create and compile the model
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error')

# Summary of the model
model.summary()

ValueError: Input shape after MaxPooling must be divisible by new_timesteps.